# Real Data Validation

In [ ]:
from jupyter_core.paths import jupyter_path
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import os
import glob
import matplotlib.pyplot as plt
from datetime import date
from datetime import datetime
from datetime import timedelta

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import TimeSeriesSplit
from sklearn import metrics
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import lightgbm as lgb # Our ML library

In [ ]:
#Cross validation libraries
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
import joblib

In [615]:
max_date = df_compact.date.max()

I've uploaded the latest SP500 data. Let's read it

In [619]:
#os.chdir("/Users/olegkazanskyi/Documents/GitHub/Trading/CSVs")
os.chdir("C:/Users/oleg.kazanskyi/Personal-oleg.kazanskyi/Trading Python/SP500_CSVs_01032023")
filepaths = [f for f in os.listdir("./") if f.endswith('.csv')]
df_val = pd.DataFrame()
for i in filepaths:
    iterate_df = pd.DataFrame()
    iterate_df = pd.read_csv(i, encoding= 'unicode_escape')
    iterate_df["stock"] = i[:-4]
    df_val = pd.concat([df_val,iterate_df])
#df = pd.concat(map(pd.read_csv, filepaths))

#os.chdir("/Users/olegkazanskyi/Documents/GitHub/Trading")
os.chdir("C:/Users/oleg.kazanskyi/Personal-oleg.kazanskyi/Trading Python")

Let's keep only the latest date we need for validation

In [635]:
df_validation = df_val[df_val.date > max_date]

Let's change the dataset with the same steps we used in this project:

In [636]:
df_validation = df_validation[df_validation.EPS_surprise.notnull()]

df_validation['DY'].fillna(0, inplace=True)
df_validation['YoY_DPR'].fillna(0, inplace=True)
df_validation['DPR'].fillna(0, inplace=True)
df_validation['YoY_DY'].fillna(0, inplace=True)     

df_validation = df_validation[df_validation.YoY_CR.notnull()]

df_validation['DE'].fillna(0, inplace=True)
df_validation['LTDE'].fillna(0, inplace=True)

df_validation = df_validation[df_validation.VIX_MoM.notnull()]

df_validation['Acc_Rec_Pay_Ration'].fillna(0, inplace=True)

df_validation = df_validation[df_validation.PEG_Backwards.notnull()]

df_validation = df_validation[df_validation.EPS_1Y_exp_Change.notnull()]

df_validation.loc[df_validation['stock'] == 'GEN','sector'] = 'Information Technology'
df_validation.loc[df_validation['stock'] == 'GEN','industry'] = 'Software & Services'

df_validation = df_validation[df_validation.YoY_AR_Ration.notnull()]

df_validation.drop(["industry"], axis = 1, inplace = True)

df_validation.replace([np.inf, -np.inf], np.nan, inplace = True)

df_validation = df_validation[df_validation.EPS_1Y_exp_Change.notnull() & df_validation.EPS_YoY_Growth.notnull() & df_validation.EPS_QoQ_frcst_diff.notnull()]

df_validation = df_validation.iloc[::5, :]

df_validation.drop(['open',
                 'close',
                 'ROA', 
                 'YoY_ROE',
                 'YoY_ROA',
                 'LTDE', 
                 'PB', 
                 'YoY_LTDE', 
                 'YoY_DE', 
                 'YoY_PB', 
                 'YoY_AR_Ration', 
                 '10YB_30MA_Vector'], axis = 1, inplace = True)

df_validation.sort_values('date', ascending = False, inplace = True)

df_validation.reset_index(inplace = True)
df_validation.drop('index', axis = 1, inplace = True)

df_validation.drop('date',axis = 1, inplace = True)

df_validation.drop('stock', axis = 1, inplace = True)

Let's label encode the sector to numeric values suitable for the LightGBM

In [622]:
df_validation["sector"] = le.transform(df_validation["sector"])

We are testing only 30 days forecast.
We can drop other target variables

In [625]:
df_validation.drop(["future_15dprice_change","future_60dprice_change","future_90dprice_change","future_120dprice_change","future_150dprice_change"], axis = 1, inplace = True)

In [630]:
df_validation = df_validation[df_validation.future_30dprice_change.notnull()]

In [632]:
lgbm = list_of_classification_models[1]

y_test = df_validation["future_30dprice_change"]
x_test = df_validation.drop('future_30dprice_change', axis = 1)

In [634]:
y_test

Series([], Name: future_30dprice_change, dtype: float64)

We can run the test of the ML model now

In [631]:
#predicting on test set
ypred=lgbm.predict(x_test)
ypred_prob = lgbm.predict_proba(x_test)[:, 1]
list_of_ypred.append(ypred)
list_of_ypred_proba.append(ypred_prob)

#calculating accuracy
precision = round(precision_score(y_test, ypred, pos_label=1, average='binary'),2)
AUC = round(roc_auc_score(y_test, ypred_prob),2)
print("\n"*3+"="*50+"\n"+f"Precision of prediction for {periods[j]}:{precision:0.2f}")
print("\n"+f"AUC of this classifier for {periods[j]}:{AUC:0.2f}")

cm = confusion_matrix(y_test, ypred)
cm_matrix = pd.DataFrame(data=cm, columns=['Predict Negative:0', 'Predict Positive:1'], 
                                 index=['Actual Negative:0', 'Actual Positive:1'])
sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu')
plt.show()

feature_imp = pd.DataFrame(sorted(zip(lgbm.feature_importances_,x_train.columns)), columns=['Value','Feature'])
feature_imp["Value, %"] = (feature_imp['Value'] / feature_imp['Value'].sum()) * 100
print(feature_imp.sort_values(by='Value, %', ascending=False)[:10])
feature_importances.append(feature_imp[["Feature", "Value, %"]])

ValueError: Input data must be 2 dimensional and non empty.